<a href="https://colab.research.google.com/github/AntonioHenriqueAC/tcc/blob/master/google_colab/Barcode_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Baixando as bibliotecas necessárias

In [0]:
!pip3 install tensorflow-gpu==1.13.1

In [0]:
!pip3 install imageai --upgrade

# 2. Baixando a CNN pré-treinada

In [0]:
!wget https://github.com/OlafenwaMoses/ImageAI/releases/download/essential-v4/pretrained-yolov3.h5

# 3. Conectando o drive ao colab


In [0]:
from google.colab import drive
drive.mount('/content/drive')

# 4. Carregando os arquivos do colab

In [0]:
!unzip "/content/drive/My Drive/Colab/datasets/barcode4.zip" 

# 5. Treinando o modelo

In [6]:
from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="barcode")
trainer.setTrainConfig(object_names_array=["barcode"], batch_size=4, num_experiments=11, train_from_pretrained_model="pretrained-yolov3.h5")
trainer.trainModel()

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

Generating anchor boxes for training images and annotation...
Average IOU for 9 anchors: 0.85
Anchor Boxes generated.
Detection configuration saved in  barcode/json/detection_config.json
Training on: 	['barcode']
Training with Batch Size:  4
Number of Experiments:  11
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Training with transfer learning from pretrained Model


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1335: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Instructions for updating:
Use tf.cast instead.
Epoch 1/11
952/952 [==============================] - 886s 930ms/step - loss: 21.8568 - yolo_layer_1_loss: 5.0254 - yolo_layer_2_loss: 5.5363 - yolo_layer_3_loss: 11.2951 - val_loss: 8.8394 - val_yolo_layer_1_loss: 3.8533 - val_yolo_layer_2_loss: 1.3593 - val_yolo_layer_3_loss: 3.6268
Epoch 2/11
952/952 [==============================] - 859s 903ms/step - loss: 8.4729 - yolo_layer_1_loss: 3.3828 - yolo_layer_2_loss: 1.0691 - yolo_layer_3_loss: 4.0210 - val_loss: 6.1117 - val_yolo_layer_1_loss: 3.9628 - val_yolo_layer_2_loss: 0.8457 - val_yolo_layer_3_loss: 1.3033
Epoch 3/11
952/952 [==============================] - 858s 901ms/step - loss: 4.9246 - yolo_layer_1_loss: 2.6632 - yolo_layer_2_loss: 0.8932 - yolo_layer_3_loss: 1.3682 - val_loss: 5.5782 - val_yolo_layer_1_loss: 3.4404 - val_yolo_layer_2_loss: 1.1977 - val_yolo_layer_3_loss: 0.9400
Epoch 4/11
952/952 [==============================] - 859s 902ms/step - loss: 3.9516 - yolo_layer_

# 6. Avaliando os modelos gerados

In [8]:
metrics = trainer.evaluateModel(model_path="barcode/models", json_path="barcode/json/detection_config.json", iou_threshold=0.5, object_threshold=0.3, nms_threshold=0.5)

Starting Model evaluation....


/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
/usr/local/lib/python3.6/dist-packages/imageai/Detection/Custom/utils/utils.py:198: RuntimeWarning: overflow encountered in exp
  h = anchors[2 * b + 1] * np.exp(h) / net_h # unit: image height
/usr/local/lib/python3.6/dist-packages/imageai/Detection/Custom/utils/utils.py:197: RuntimeWarning: overflow encountered in exp
  w = anchors[2 * b + 0] * np.exp(w) / net_w # unit: image width


skipping the evaluation of barcode/models/detection_model-ex-001--loss-0021.857.h5 because following exception occurred: cannot convert float infinity to integer
Model File:  barcode/models/detection_model-ex-002--loss-0008.473.h5 

Using IoU :  0.5
Using Object Threshold :  0.3
Using Non-Maximum Suppression :  0.5
barcode: 0.4783
mAP: 0.4783
Model File:  barcode/models/detection_model-ex-003--loss-0004.925.h5 

Using IoU :  0.5
Using Object Threshold :  0.3
Using Non-Maximum Suppression :  0.5
barcode: 0.5783
mAP: 0.5783
Model File:  barcode/models/detection_model-ex-004--loss-0003.952.h5 

Using IoU :  0.5
Using Object Threshold :  0.3
Using Non-Maximum Suppression :  0.5
barcode: 0.5483
mAP: 0.5483
Model File:  barcode/models/detection_model-ex-005--loss-0003.563.h5 

Using IoU :  0.5
Using Object Threshold :  0.3
Using Non-Maximum Suppression :  0.5
barcode: 0.6950
mAP: 0.6950
Model File:  barcode/models/detection_model-ex-006--loss-0003.316.h5 

Using IoU :  0.5
Using Object Thres

# 7. Criando pastas


In [0]:
import os

path, dirs, files = next(os.walk("/content/drive/My Drive/Colab/Corridas"))
file_count_corridas = len(files)
print("Corridas: ", file_count_corridas + 1)

file_count_corridas = file_count_corridas+1
newFolderCorrida = file_count_corridas + 1
newFolderCorrida = str(newFolderCorrida)
folder = "/content/drive/My\ Drive/Colab/Corridas/Corrida_" + newFolderCorrida
!mkdir $folder


# 8. Setando as variaveis


In [0]:
modelPath = "/content/barcode/models/detection_model-ex-008--loss-0002.715.h5"
JsonPath = "drive/My Drive/Colab/json/detection_config_barcode.json"
InputImage = "drive/My Drive/Colab/Imagens_Originais/img3.jpg"

# 9. Treinando modelo e recortando os codigos de barras da imagem original


In [0]:
from imageai.Detection.Custom import CustomObjectDetection

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath(modelPath)
detector.setJsonPath(JsonPath)
detector.loadModel()
detections, extracted_objects_array = detector.detectObjectsFromImage(input_image = InputImage, 
                                                                      output_image_path="barcode-detected.jpg",
                                                                      minimum_percentage_probability=10,
                                                                      extract_detected_objects=True)
def boxPoints(e):
  return e['box_points']

detections.sort(key=boxPoints)

print("Número       " , ": ", "Porcentagem ", "      :    ", "box_points")
print("----------------------------------------------------------------")

print("")

for detection, object_path in zip(detections, extracted_objects_array):
    print(detection["name"], "      : ", detection["percentage_probability"], " : ", detection["box_points"])


print("----------------------------------------------------------------")
print("Quantidade de código de barras detectados na imagem: ", len(detections))

In [0]:
  def removeDuplicates():
      i=0
      soma=0
      for detection in detections:
        if( i != len(detections)-1 ):
            
            Xa = (detections[i]['box_points'][0] + detections[i]['box_points'][2])/2
            Ya = (detections[i]['box_points'][1] + detections[i]['box_points'][3])/2
            Xb = (detections[i+1]['box_points'][0] + detections[i+1]['box_points'][2])/2
            Yb = (detections[i+1]['box_points'][1] + detections[i+1]['box_points'][3])/2

            Dab = ( (Xb-Xa)**2 + (Yb - Ya)**2 )**1/2
            if(Dab < float(10)):
                if(detections[i]["percentage_probability"] < detections[i+1]["percentage_probability"]):
                    detections.pop(i)
                else:
                    detections.pop(i+1)

        soma = soma + detections[i]["percentage_probability"]
        i=i+1

# 10. Movendo as pastas para o diretório correto.


In [0]:
!mv /content/barcode-detected.jpg $folder
!mv /content/barcode-detected-objects $folder

barcode = "/content/drive/My\ Drive/Colab/Corridas/Corrida_" + newFolderCorrida + "/barcode-detected-objects"
images = "/content/drive/My\ Drive/Colab/Corridas/Corrida_" + newFolderCorrida + "/images_tags"
!mv  $barcode $images